<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/icp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

# Import necessary utils

In [ ]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
from pyflann import *

# Run a first example of flann search
In this exercise we will exploy the [FLANN Library](https://pypi.org/project/flann/) a Fast Library for Approximate Nearest Neighbors (FLANN)

In [ ]:
# a first example
dataset = np.random.rand(10000, 128)
testset = np.random.rand(1000, 128)
flann = FLANN()
[result, dists] = flann.nn(dataset, testset, 1, algorithm="kmeans", branching=32, iterations=7, checks=16)
result.shape

# Load shape and prepare the GT tranformation

In [ ]:
# read mesh
v, f = igl.read_triangle_mesh(os.path.join('.', "data", "MooseOBJ.obj"))
v1 = v;
H = np.random.randn(n, n)
Rot_gt, tmp = qr(H)
Trasl_gt = np.random.randn(1, 3)
v2 = np.transpose(Rot_gt*v.T + Trasl_gt)

# check that R_gt is a rotation.

# plot v1 and v2

# implement ICP
Following the slides from our this course we can implement the ICP between v1 and v2. You can also take a look at the [following paper](http://graphics.stanford.edu/~smr/ICP/comparison/eggert_comparison_mva97.pdf) (Section 3.1) for a more in-depth discussion of the computation.

In [ ]:
# initialize some variables
Stilde = np.diag([1,1,-1]);
X = v1;
Y = v2;
nv = X.shape[0]

muX = np.transpose((1/nv)*np.sum(X))
muY = np.transpose((1/nv)*np.sum(X))
[idxs, dists] = flann.nn(Y, X, 1, algorithm="kmeans", branching=32, iterations=7, checks=16)

while np.amax(dists) > 1e-10:
   C = (Y[idxs,:] - muY)*np.transpose(X - muX)
   [U, S, V] = np.linalg.svd(C, full_matrices=True)
   Ropt = U*V;
   if numpy.linalg.det(Ropt)<0:
       Ropt = U*Stilde*V 

   Topt = np.transpose(np.transpose(muY) - Ropt*np.transpose(muX))
   
   X = np.transpose(np.matmul(Ropt,np.transpose(X)) + Topt;
   [idxs, dists] = flann.nn(Y, X, 1, algorithm="kmeans", branching=32, iterations=7, checks=16)
   muX = np.transpose((1/nv)*np.sum(X))
   # plot update